In [70]:
import pandas as pd
import random
import json
import re
from datetime import datetime, timedelta, timezone
import subprocess

In [71]:

# =========================
# CONFIGURATION
# =========================
SEED = 42
START_DATE = "2025-06-01"
NUM_DAYS = 1
PATIENT_LIMIT = 1
random.seed(SEED)


In [72]:

# =========================
# FUNCTIONS
# =========================

def query_ollama(model: str, prompt: str) -> str:
    """Send a prompt to Ollama and get text output."""
    try:
        result = subprocess.run(
            ["ollama", "run", model],
            input=prompt,
            text=True,
            capture_output=True,
            check=True
        )
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print("Ollama error:", e.stderr)
        return "[]"

def generate_observation_windows(start_date, num_days):
    windows = []
    base = datetime.strptime(start_date, "%Y-%m-%d").replace(tzinfo=timezone.utc)
    for day in range(num_days):
        for start_hour in [0, 6, 12, 18]:
            start = base + timedelta(days=day, hours=start_hour)
            end = start + timedelta(hours=6)
            windows.append((start.isoformat(), end.isoformat()))
    return windows


In [73]:

# ---------- Vitals ----------
def random_vitals(state="normal"):
    if state == "normal":
        hr = random.randint(65, 80)
        spo2 = random.randint(95, 99)
        temp = round(random.uniform(36.4, 37.0), 1)
        sys_bp = random.randint(110, 130)
        dia_bp = random.randint(70, 80)
    elif state == "uncomfortable":
        hr = random.randint(75, 90)
        spo2 = random.randint(92, 97)
        temp = round(random.uniform(37.0, 37.5), 1)
        sys_bp = random.randint(130, 140)
        dia_bp = random.randint(80, 85)
    else:  # sick
        hr = random.randint(90, 110)
        spo2 = random.randint(88, 93)
        temp = round(random.uniform(37.6, 38.5), 1)
        sys_bp = random.randint(140, 150)
        dia_bp = random.randint(85, 95)

    return {
        "heartRate": {"value": hr, "unit": "bpm"},
        "spo2": {"value": spo2, "unit": "%"},
        "temperature": {"value": temp, "unit": "°C"},
        "bloodPressure": {"systolic": sys_bp, "diastolic": dia_bp, "unit": "mmHg"},
    }


In [74]:
# ---------- Medications ----------
from datetime import datetime, timedelta

def suggest_medications_with_ollama(vitals, obs_start):
    prompt = f"""
You are a medical assistant. 

Patient vitals:
{json.dumps(vitals, indent=2)}

Observation start: {obs_start}

Suggest 1–2 relevant general medications with:
- name
- dose in mg (always include unit, e.g., "75mg")
- complianceStatus (Taken, Missed, Delayed)

Do not include scheduledTime — it will be added later.
Return as a JSON array.
"""
    response = query_ollama("gemma3:4b", prompt)
    print("Raw Ollama response for meds:", response)

    cleaned = re.sub(r"^```json|```|'|\"\"\"", "", response, flags=re.IGNORECASE).strip()

    try:
        meds = json.loads(cleaned)

        # Parse observation start time
        obs_dt = datetime.fromisoformat(obs_start.replace("Z", "+00:00"))

        # Assign realistic scheduled times within 4h window
        for med in meds:
            if "mg" not in med["dose"]:
                med["dose"] = f"{med['dose']}mg"

            rand_minutes = random.randint(0, 240)  # 0–240 min (4h)
            sched_time = obs_dt + timedelta(minutes=rand_minutes)
            med["scheduledTime"] = sched_time.isoformat().replace("+00:00", "Z")

        print(f"✅ Suggested {len(meds)} medications with scheduled times (within 4h of {obs_start}).")

    except json.JSONDecodeError:
        print("⚠️ Ollama response could not be parsed, using empty meds list")
        meds = []

    return meds


In [75]:
# ---------- Nursing Note ----------
def generate_nursing_note_with_ollama(vitals, medications, obs_start, state="normal"):
    # Tailor instructions based on patient state
    if state == "normal":
        condition_instruction = "Summarize the patient as stable, alert, and comfortable."
    elif state == "uncomfortable":
        condition_instruction = "Indicate mild discomfort, slight deviations in vitals, and advise continued monitoring."
    else:  # sick
        condition_instruction = "Document the patient as unwell with notable abnormalities; recommend closer observation and possible escalation."

    prompt = f"""
You are an experienced nurse documenting a patient observation. 
Write a concise, professional, and natural-sounding nursing note.

- Use 3–4 sentences only.  
- Summarize the patient's condition based on vitals, using natural language rather than repeating raw numbers.  
- Mention medications administered clearly and naturally.  
- {condition_instruction}  
- Vary sentence structure to avoid repetitive wording.  
- Tone must be factual and objective; do not include personal commentary.  

Patient vitals:
{json.dumps(vitals, indent=2)}

Medications administered:
{json.dumps(medications, indent=2)}

Observation start: {obs_start}

Return plain text only, no markdown, no JSON, no headers.
"""
    response = query_ollama("nursing_note_generator", prompt)
    print("Raw Ollama response for nursing note:", response)

    # Clean the response
    note = re.sub(r"^```.*|```$", "", response, flags=re.MULTILINE).strip()
    note = re.sub(r"^(Nursing Note:|Output:)\s*", "", note, flags=re.IGNORECASE).strip()

    # Fallback note in case Ollama output is empty
    if not note:
        fallback_templates = {
            "normal": [
                "Patient appears stable and comfortable. Vitals are within expected ranges. Medications administered as prescribed.",
                "Observation shows patient alert with stable vitals. Medications given as scheduled. No concerns noted.",
                "Patient remains well, vitals normal. All prescribed medications were taken without issue."
            ],
            "uncomfortable": [
                "Patient shows mild discomfort with slight fluctuations in vitals. Medications administered as ordered. Monitoring advised.",
                "Observation indicates patient uneasy; vitals mildly irregular. Medications given. Continued supervision recommended.",
                "Patient exhibits mild symptoms, vitals slightly outside normal range. Medications taken as scheduled."
            ],
            "sick": [
                "Patient presents with abnormal vitals and appears unwell. Medications administered. Close monitoring required.",
                "Observation reveals patient in poor condition with significant vital deviations. Medications given as prescribed. Escalation may be necessary.",
                "Patient condition unstable with notable abnormalities in vitals. Medications administered, ongoing observation needed."
            ]
        }
        note = random.choice(fallback_templates[state])

    print("✅ Final Nursing Note:", note)
    return note


In [76]:
# ---------- ADLs ----------
def random_adls(state="normal"):
    if state == "normal":
        return {
            "stepsTaken": random.randint(500, 2000),   # ~2k–8k per day → scaled for 6h
            "calorieIntake": random.randint(400, 900), # One or two meals/snacks
            "sleepHours": round(random.uniform(0, 2), 1), # Could nap within 6h
            "waterIntakeMl": random.randint(300, 800), # A few glasses
            "mealsSkipped": random.choices([0, 1], [0.8, 0.2])[0],
            "exerciseMinutes": random.randint(5, 20),  # Short sessions possible
            "bathroomVisits": random.randint(1, 3),    # Typical 6h range
        }
    elif state == "uncomfortable":
        return {
            "stepsTaken": random.randint(200, 1000),
            "calorieIntake": random.randint(200, 600),
            "sleepHours": round(random.uniform(0, 1), 1),
            "waterIntakeMl": random.randint(200, 500),
            "mealsSkipped": random.choices([0, 1], [0.5, 0.5])[0],
            "exerciseMinutes": random.randint(0, 10),
            "bathroomVisits": random.randint(2, 5),    # Could be slightly irregular
        }
    else:  # sick
        return {
            "stepsTaken": random.randint(0, 400),
            "calorieIntake": random.randint(100, 400),
            "sleepHours": round(random.uniform(1, 3), 1), # Might rest more
            "waterIntakeMl": random.randint(100, 400),
            "mealsSkipped": random.choices([1, 2], [0.6, 0.4])[0],
            "exerciseMinutes": 0,
            "bathroomVisits": random.randint(3, 7),    # Higher in illness (e.g. GI upset)
        }


In [77]:
# ---------- Behaviour Tags ----------
def generate_behaviour_tags(state="normal", vitals=None, medications=None, adls=None):
    tags = []

    # Base state-driven probabilities
    if state == "normal":
        if random.random() < 0.8: tags.append("cooperative")
        if random.random() < 0.2: tags.append("talkative")
    elif state == "uncomfortable":
        if random.random() < 0.6: tags.append("refusedActivity")
        if random.random() < 0.5: tags.append("poorDiet")
        if random.random() < 0.5: tags.append("agitated")
        if random.random() < 0.3: tags.append("restless")
        if random.random() < 0.3: tags.append("anxious")
    else:  # sick
        if random.random() < 0.7: tags.append("refusedActivity")
        if random.random() < 0.6: tags.append("poorDiet")
        if random.random() < 0.5: tags.append("noncompliantMedication")
        if random.random() < 0.6: tags.append("agitated")
        if random.random() < 0.4: tags.append("weak")
        if random.random() < 0.4: tags.append("lethargic")
        if random.random() < 0.3: tags.append("confused")

    # Vitals-driven tags
    if vitals:
        if vitals["temperature"]["value"] > 38.0:
            tags.append("feverish")
            tags.append("restless")
        if vitals["heartRate"]["value"] > 110:
            tags.append("anxious")
            tags.append("agitated")
        if vitals["heartRate"]["value"] < 55:
            tags.append("dizzy")
            tags.append("lethargic")
        if vitals["spo2"]["value"] < 92:
            tags.append("weak")
            tags.append("fatigued")
        if vitals["bloodPressure"]["systolic"] > 150 or vitals["bloodPressure"]["diastolic"] > 95:
            tags.append("restless")
            tags.append("irritable")

    # Medication-driven tags
    if medications:
        for med in medications:
            name = med.get("name", "").lower()
            status = med.get("complianceStatus")
            if status == "Missed":
                tags.append("noncompliantMedication")
            if any(pain in name for pain in ["pain", "ibuprofen", "acetaminophen"]):
                tags.append("inPain")
                tags.append("complaining")
            if any(anti in name for anti in ["anti", "nausea"]):
                tags.append("nauseous")

    # ADLs-driven tags
    if adls:
        if adls.get("stepsTaken",0) < 500:
            tags.append("lowMobility")
        if adls.get("sleepHours",0) < 2:
            tags.append("tired")
            tags.append("sleepy")
        if adls.get("calorieIntake",0) < 200:
            tags.append("poorNutrition")
        if adls.get("waterIntakeMl",0) < 200:
            tags.append("dehydrated")
        if adls.get("mealsSkipped",0) > 0:
            tags.append("skippedMeals")
        if adls.get("exerciseMinutes",0) == 0:
            tags.append("inactive")

    # Remove duplicates
    return list(set(tags))


In [78]:

# ---------- Emotion Analysis ----------
def generate_emotion_analysis(state="normal"):
    if state == "normal":
        return {"tags": [], "confidenceScores": {}}
    elif state == "uncomfortable":
        return {
            "tags": ["frustrated"],
            "confidenceScores": {"frustrated": round(random.uniform(0.6, 0.85), 2)},
        }
    else:
        return {
            "tags": ["frustrated", "lowEnergy"],
            "confidenceScores": {
                "frustrated": round(random.uniform(0.75, 0.95), 2),
                "lowEnergy": round(random.uniform(0.7, 0.9), 2),
            },
        }


In [79]:
import random

# ---------- Symptom Inference ----------
def infer_symptoms(vitals, medications, adls, nursing_note):
    """Infer likely symptoms from vitals, medications, ADLs, and nursing note with realistic probabilities."""
    symptoms = []

    # Vitals-based inference
    if vitals["temperature"]["value"] > 37.5 and random.random() < 0.9:
        symptoms.append("fever")
    if vitals["heartRate"]["value"] < 60 and random.random() < 0.7:
        symptoms.append("bradycardia")
    elif vitals["heartRate"]["value"] > 100 and random.random() < 0.8:
        symptoms.append("tachycardia")
    if vitals["bloodPressure"]["systolic"] > 140 or vitals["bloodPressure"]["diastolic"] > 90:
        if random.random() < 0.8:
            symptoms.append("high blood pressure")
    if vitals["spo2"]["value"] < 94 and random.random() < 0.9:
        symptoms.append("low oxygen saturation")

    # Medication-based hints
    for med in medications:
        name_lower = med["name"].lower()
        if any(x in name_lower for x in ["pain", "ibuprofen", "acetaminophen"]) and random.random() < 0.9:
            symptoms.append("pain")
        if any(x in name_lower for x in ["anti", "nausea"]) and random.random() < 0.7:
            symptoms.append("nausea")

    # ADLs-based hints
    if adls["stepsTaken"] < 500 and random.random() < 0.8:
        symptoms.append("low mobility")
    if adls["sleepHours"] < 4 and random.random() < 0.8:
        symptoms.append("fatigue")
    if adls["calorieIntake"] < 1200 and random.random() < 0.7:
        symptoms.append("poor nutrition")
    if adls["waterIntakeMl"] < 800 and random.random() < 0.6:
        symptoms.append("dehydration")
    if adls["mealsSkipped"] > 0 and random.random() < 0.6:
        symptoms.append("missed meals")

    # Nursing note keyword extraction with probability
    keywords = ["pain", "headache", "discomfort", "weakness", "dizzy", "tired", "nausea"]
    for kw in keywords:
        if kw in nursing_note.lower() and random.random() < 0.9:
            symptoms.append(kw)

    # Remove duplicates
    return list(set(symptoms))


In [80]:

# ---------- Clinical Summary with Ollama ----------
def generate_clinical_summary_with_ollama(vitals, medications, adls, nursing_note, obs_start):
    symptoms = infer_symptoms(vitals, medications, adls, nursing_note)

    prompt = f"""
You are a clinical assistant. Write a concise clinical summary for a patient observation.

Vitals:
{json.dumps(vitals, indent=2)}

Medications:
{json.dumps(medications, indent=2)}

ADLs:
{json.dumps(adls, indent=2)}

Nursing note:
{nursing_note}

Observed symptoms:
{json.dumps(symptoms)}

Observation start: {obs_start}

Return a single concise clinical summary (1–2 sentences), mentioning relevant vitals, symptoms, and medications. Do not include any preamble or explanations. Plain text only.
"""
    response = query_ollama("gemma3:4b", prompt)
    # Clean any ``` fences
    summary = re.sub(r"^```|```$", "", response).strip()
    print("✅ Generated clinical summary with Ollama:", summary)
    return summary


In [81]:

# ---------- Generate Patient Observations ----------
def generate_patient_observations(patient, start_date, num_days):
    windows = generate_observation_windows(start_date, num_days)
    records = []

    for obs_start, obs_end in windows:
        # Determine state based on probabilities
        state = random.choices(
            ["normal", "uncomfortable", "sick"], weights=[0.7, 0.2, 0.1], k=1
        )[0]

        vitals = random_vitals(state)
        medications = suggest_medications_with_ollama(vitals, obs_start)
        nursing_note = generate_nursing_note_with_ollama(vitals, medications, obs_start,state)
        adls = random_adls(state)
        record = {
            "patientId": patient["patient_id"],
            "age": int(patient["age"]),
            "gender": patient["gender"],
            "observationStart": obs_start,
            "observationEnd": obs_end,
            "nursingNote": nursing_note,
            "medications": medications,
            "vitals": vitals,
            "adls": adls,
            "behaviourTags": generate_behaviour_tags(state,vitals,medications,adls),
            "emotionAnalysis": generate_emotion_analysis(state),
            "clinicalSummary": generate_clinical_summary_with_ollama(vitals, medications, adls,nursing_note,state),
            "entitiesExtracted": {"symptoms": [], "vitals": [], "procedures": [], "medications": []},
            "baselineStats": {
                "avgHeartRate": round(random.uniform(70, 85), 1),
                "avgSpo2": round(random.uniform(95, 98), 1),
                "avgSleepHours": round(random.uniform(5, 8), 1),
                "avgCalorieIntake": round(random.uniform(1600, 2200), 1),
                "avgScreenTimeMinutes": round(random.uniform(30, 180), 1),
                "avgBathroomVisits": random.randint(2, 6),
                "usualDietCompliance": round(random.uniform(0.7, 1.0), 2)
            },
            "alerts": []
        }
        print(f"Generated observation for patient {patient['patient_id']} ({state})")
        records.append(record)
    return records


In [82]:

# =========================
# MAIN SCRIPT
# =========================

if __name__ == "__main__":
    patients = pd.read_csv("patients.csv").head(PATIENT_LIMIT)
    all_records = []

    for _, patient in patients.iterrows():
        all_records.extend(generate_patient_observations(patient, START_DATE, NUM_DAYS))

    with open("patient_observations.json", "w") as f:
        json.dump(all_records, f, indent=2)

    print(f"Generated {len(all_records)} observations for {len(patients)} patients.")


python(40530) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Raw Ollama response for meds: ```json
[
  {
    "name": "Acetaminophen",
    "dose": "500mg",
    "complianceStatus": "Taken"
  },
  {
    "name": "Folic Acid",
    "dose": "1mg",
    "complianceStatus": "Taken"
  }
]
```
✅ Suggested 2 medications with scheduled times (within 4h of 2025-06-01T00:00:00+00:00).


python(40531) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Raw Ollama response for nursing note: Patient remains stable with a heart rate of 65 bpm and oxygen saturation at 97%. Blood pressure was 114/71 mmHg, and the temperature was 36.5°C. Acetaminophen 500mg was administered, and folic acid 1mg was also taken as prescribed. The patient reports being comfortable and is alert.
✅ Final Nursing Note: Patient remains stable with a heart rate of 65 bpm and oxygen saturation at 97%. Blood pressure was 114/71 mmHg, and the temperature was 36.5°C. Acetaminophen 500mg was administered, and folic acid 1mg was also taken as prescribed. The patient reports being comfortable and is alert.


python(40533) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

Raw Ollama response for meds: ```json
[
  {
    "name": "Acetaminophen",
    "dose": "500mg",
    "complianceStatus": "Taken"
  },
  {
    "name": "Ondansetron",
    "dose": "4mg",
    "complianceStatus": "Taken"
  }
]
```
✅ Suggested 2 medications with scheduled times (within 4h of 2025-06-01T18:00:00+00:00).


python(40550) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Raw Ollama response for nursing note: Patient remains stable with a heart rate of 75 bpm and oxygen saturation at 96%. Blood pressure was 122/80 mmHg, and the temperature was 36.8°C. Acetaminophen 500mg and Ondansetron 4mg were administered as prescribed. The patient reports being comfortable and is alert.
✅ Final Nursing Note: Patient remains stable with a heart rate of 75 bpm and oxygen saturation at 96%. Blood pressure was 122/80 mmHg, and the temperature was 36.8°C. Acetaminophen 500mg and Ondansetron 4mg were administered as prescribed. The patient reports being comfortable and is alert.


python(40551) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


✅ Generated clinical summary with Ollama: Patient remains stable with a heart rate of 75 bpm and oxygen saturation at 96%, presenting with fatigue, dehydration, and poor nutrition. Acetaminophen and Ondansetron were administered as prescribed.
Generated observation for patient P0001 (normal)
Generated 4 observations for 1 patients.


In [83]:
import requests
import json

# URL of your Google Apps Script Web App
url = "https://script.google.com/macros/s/AKfycbyPOXMMnMb4kC5lbGEnqirRph_HcsLFgyfqWuDnNDmqEq9QDHnLKWNvN7_MvPks2A-v-w/exec"

# Load your generated observations
with open("patient_observations.json") as f:
    data = json.load(f)

# Send POST request
response = requests.post(url, json=data)

# Check response
print("Status code:", response.status_code)
print("Response text:", response.text)


Status code: 200
Response text: {"status":"success","message":"Data inserted"}
